In [ ]:
#Mounting drive
from google.colab import drive
drive.mount('/content/drive')


In [11]:
import torch
torch.cuda.set_device(0)
print(torch.__version__)
print(torch.version.cuda)
print(torch.cuda.is_available())

# Print memory reserved and allocated
print(torch.cuda.memory_reserved())
print(torch.cuda.memory_allocated())

1.13.1
11.7
True
5502926848
5263157760


In [2]:
pip install -r ~/Documents/inseer_ML_Project/yolov8/requirements.txt


Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install ultralytics


Note: you may need to restart the kernel to use updated packages.


In [3]:
from ultralytics import YOLO
import cv2
from pathlib import Path
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import os
from types import SimpleNamespace
import time
from sklearn.model_selection import train_test_split
from zipfile import ZipFile
import shutil

In [2]:
!pip install scikit-learn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 70.2 MB/s eta 0:00:00m eta 0:00:01:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 45.8 MB/s eta 0:00:00


In [5]:
zip_file_path = '/content/drive/Shareddrives/Nikhil shared drive/COCO-Hand.zip'
extraction_path = '/content/data/'

In [ ]:
# Unzip the file
with ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extraction_path)

print("Unzipped successfully!")

In [ ]:
#Splitting the dataset into train,test and val sets
def split_images(images_dir, output_dir):
    """
    Splits images into train, val, and test sets and move them to respective folders.
    """

    # List all image filenames
    images = [f for f in os.listdir(images_dir) if f.endswith('.jpg')]

    # Split the filenames into train, val, and test sets
    train_images, temp_images = train_test_split(images, test_size=0.2, random_state=42)
    val_images, test_images = train_test_split(temp_images, test_size=0.5, random_state=42)

    # Dictionaries to map split name to filenames
    splits = {
        'train': train_images,
        'val': val_images,
        'test': test_images
    }

    # Create directories for each split and move images to them
    for split, split_images in splits.items():
        split_dir = Path(output_dir) / split / 'images'
        split_dir.mkdir(parents=True, exist_ok=True)
        for img in split_images:
            shutil.move(images_dir / img, split_dir / img)

    return splits

# Test the function
images_dir = Path(os.path.expanduser("~/Documents/Datasets/COCO dataset/COCO-Hand/COCO-Hand-Big/COCO-Hand-Big_Images"))
output_dir = Path(os.path.expanduser("~/Documents/Datasets/COCO dataset/COCO-Hand-splitdata"))

# Call the function to split images
splits = split_images(images_dir, output_dir)



In [ ]:
def load_annotations_from_file(annotation_file_path):
    """
    Load annotations from the specified file.
    """
    with open(annotation_file_path, 'r') as file:
        annotations = [line.strip().split() for line in file.readlines()]
    return annotations

annotations = load_annotations_from_file(Path(os.path.expanduser("~/Documents/Datasets/COCO dataset/COCO-Hand/COCO-Hand-Big/COCO-Hand-Big_annotations.txt")))
print(f"Total annotations loaded: {len(annotations)}")

In [3]:
# Define paths for each split
train_images_path = Path(os.path.expanduser("~/Documents/Datasets/COCO dataset/COCO-Hand-splitdata/train/images"))
val_images_path = Path(os.path.expanduser("~/Documents/Datasets/COCO dataset/COCO-Hand-splitdata/val/images"))
test_images_path = Path(os.path.expanduser("~/Documents/Datasets/COCO dataset/COCO-Hand-splitdata/test/images"))

train_labels_path = Path(os.path.expanduser("~/Documents/Datasets/COCO dataset/COCO-Hand-splitdata/train/labels"))
val_labels_path = Path(os.path.expanduser("~/Documents/Datasets/COCO dataset/COCO-Hand-splitdata/val/labels"))
test_labels_path = Path(os.path.expanduser("~/Documents/Datasets/COCO dataset/COCO-Hand-splitdata/test/labels"))




In [ ]:
#path for the annotations file
annotations_file_path =Path(os.path.expanduser("~/Documents/Datasets/COCO dataset/COCO-Hand/COCO-Hand-Big/COCO-Hand-Big_annotations.txt"))



# Load annotations from the provided file
with open(annotations_file_path, 'r') as file:
    annotations_data = [line.strip() for line in file.readlines()]


# Define paths for each set
train_images_path = Path(os.path.expanduser("~/Documents/Datasets/COCO dataset/COCO-Hand-splitdata/train/images"))
val_images_path = Path(os.path.expanduser("~/Documents/Datasets/COCO dataset/COCO-Hand-splitdata/val/images"))
test_images_path = Path(os.path.expanduser("~/Documents/Datasets/COCO dataset/COCO-Hand-splitdata/test/images"))


# Get image filenames from each directory and combine them
train_image_filenames = [f for f in os.listdir(train_images_path) if f.endswith('.jpg')]
val_image_filenames = [f for f in os.listdir(val_images_path) if f.endswith('.jpg')]
test_image_filenames = [f for f in os.listdir(test_images_path) if f.endswith('.jpg')]


test_relevant_annotations = [ann for ann in annotations_data if any(img in ann for img in test_image_filenames)]
train_relevant_annotations = [ann for ann in annotations_data if any(img in ann for img in train_image_filenames)]

val_relevant_annotations = [ann for ann in annotations_data if any(img in ann for img in val_image_filenames)]

print(val_relevant_annotations)
print(len(train_relevant_annotations))


In [ ]:
from PIL import Image

def convert_to_yolo_format(annotation, image_path):
    """
    Convert COCO-Hand annotation to YOLO format.
    """
    # Extract bounding box coordinates
    parts = annotation.split(',')
    image_name = parts[0]
    xmin, xmax, ymin, ymax = map(float, parts[1:5])

    # Calculate absolute width and height
    width_abs = abs(xmax - xmin)
    height_abs = abs(ymax - ymin)

    # Load the image to get its width and height
    with Image.open(image_path) as img:
        img_width, img_height = img.size

    # Calculate YOLO format values
    x_center = ((xmin + xmax) / 2) / img_width
    y_center = ((ymin + ymax) / 2) / img_height
    width = width_abs / img_width
    height = height_abs / img_height

    return [0, x_center, y_center, width, height]  # 0 is the class_id for 'hand'



In [ ]:

# Creating a dictionary to store annotations for each image for TEST dataset
print(test_relevant_annotations)
test_annotations_dict = {}
for ann in test_relevant_annotations:
    image_name = ann.split(',')[0]
    if image_name not in test_annotations_dict:
        test_annotations_dict[image_name] = []
    test_annotations_dict[image_name].append(ann)
print(test_annotations_dict)

In [ ]:
#Create a dictionary to store annotations for each image for train dataset
train_annotations_dict = {}
for ann in train_relevant_annotations:
    image_name = ann.split(',')[0]
    if image_name not in train_annotations_dict:
        train_annotations_dict[image_name] = []
    train_annotations_dict[image_name].append(ann)
print(len(train_annotations_dict))

In [ ]:
#Create a dictionary to store annotations for each image for val dataset
val_annotations_dict = {}
for ann in val_relevant_annotations:
    image_name = ann.split(',')[0]
    if image_name not in val_annotations_dict:
        val_annotations_dict[image_name] = []
    val_annotations_dict[image_name].append(ann)
print(val_annotations_dict)

In [ ]:
# Convert each annotation to YOLO format and save to .txt file for TEST IMAGES



for image_name, image_annotations in test_annotations_dict.items():
    yolo_formatted_annotations = [convert_to_yolo_format(ann, os.path.join(test_images_path, image_name)) for ann in image_annotations]  # Use image_annotations instead of test_relevant_annotations
    yolo_strings = [" ".join(map(str, ann)) for ann in yolo_formatted_annotations]

    txt_path = os.path.join(test_labels_path, image_name.replace('.jpg', '.txt'))

    labels_directory = os.path.dirname(txt_path)
    if not os.path.exists(labels_directory):
      os.makedirs(labels_directory)

    with open(txt_path, 'w') as f:
        f.write("\n".join(yolo_strings))


In [ ]:

# Convert each annotation to YOLO format and save to .txt file FOR TRAIN IMAGES
for image_name, image_annotations in train_annotations_dict.items():
    yolo_formatted_annotations = [convert_to_yolo_format(ann, os.path.join(train_images_path, image_name)) for ann in image_annotations]
    yolo_strings = [" ".join(map(str, ann)) for ann in yolo_formatted_annotations]

    txt_path = os.path.join(train_labels_path, image_name.replace('.jpg', '.txt'))

    labels_directory = os.path.dirname(txt_path)
    if not os.path.exists(labels_directory):
      os.makedirs(labels_directory)

    with open(txt_path, 'w') as f:
        f.write("\n".join(yolo_strings))

In [ ]:
# Convert each annotation to YOLO format and save to .txt file FOR VAL IMAGES
for image_name, image_annotations in val_annotations_dict.items():
    yolo_formatted_annotations = [convert_to_yolo_format(ann, os.path.join(val_images_path, image_name)) for ann in image_annotations]
    yolo_strings = [" ".join(map(str, ann)) for ann in yolo_formatted_annotations]

    txt_path = os.path.join(val_labels_path, image_name.replace('.jpg', '.txt'))

    labels_directory = os.path.dirname(txt_path)
    if not os.path.exists(labels_directory):
      os.makedirs(labels_directory)

    with open(txt_path, 'w') as f:
        f.write("\n".join(yolo_strings))

In [ ]:
#Creating the YAML file
def create_yaml_configuration():
    """
    Creates a .YAML configuration file for training YOLO.
    Returns the path to the created .YAML file.
    """
    # Define the content for the .YAML configuration file
    yaml_content = """
    train: /home/nikhil/Documents/Datasets/COCO dataset/COCO-Hand-splitdata/train
    test: /home/nikhil/Documents/Datasets/COCO dataset/COCO-Hand-splitdata/test
    val: /home/nikhil/Documents/Datasets/COCO dataset/COCO-Hand-splitdata/val

    # Classes
    nc: 1
    names: ['hand']
    """

    # Define the path for the .YAML file
    yaml_path = Path(os.path.expanduser("~/Documents/Datasets/COCO dataset/COCO-Hand-splitdata/coco_hand_big.yaml"))

    # Write the content to the .YAML file
    with open(yaml_path, 'w') as yaml_file:
        yaml_file.write(yaml_content)

    # Return the path to the created .YAML file
    return yaml_path

create_yaml_configuration()


In [7]:
#Train model
def train_model(data_yaml):

    model = YOLO("yolov8m.pt")
    model.train(data=data_yaml, epochs=100, project="COCO-Hand", name="COCO_Hand_Big", imgsz=640)
    model.val()
    
train_model('/home/nikhil/Documents/Datasets/COCO dataset/COCO-Hand-splitdata/coco_hand_big.yaml')

Ultralytics YOLOv8.0.208 🚀 Python-3.9.7 torch-1.13.1 CUDA:0 (NVIDIA TITAN RTX, 24220MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/home/nikhil/Documents/Datasets/COCO dataset/COCO-Hand-splitdata/coco_hand_big.yaml, epochs=100, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=COCO-Hand, name=COCO_Hand_Big2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None,

 ········


wandb: Appending key for api.wandb.ai to your netrc file: /home/nikhil/.netrc


Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
100%|██████████| 6.23M/6.23M [00:00<00:00, 72.9MB/s]
AMP: checks failed ❌. Anomalies were detected with AMP on your system that may lead to NaN losses or zero-mAP results, so AMP will be disabled during training.
train: Scanning /home/nikhil/Documents/Datasets/COCO dataset/COCO-Hand-splitdata/train/labels.cache... 21199 images, 0 backgrounds, 197 corrupt: 100%|██████████| 21199/21199 [00:00<?, ?it/s]
train: WARNING ⚠️ /home/nikhil/Documents/Datasets/COCO dataset/COCO-Hand-splitdata/train/images/000000000036.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0069]
train: WARNING ⚠️ /home/nikhil/Documents/Datasets/COCO dataset/COCO-Hand-splitdata/train/images/000000001668.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0028]
train: WARNING ⚠️ /home/nikhil/Documents/Datasets/COCO dataset/COCO-Hand-splitdata

val: Scanning /home/nikhil/Documents/Datasets/COCO dataset/COCO-Hand-splitdata/val/labels.cache... 2650 images, 0 backgrounds, 15 corrupt: 100%|██████████| 2650/2650 [00:00<?, ?it/s]
val: WARNING ⚠️ /home/nikhil/Documents/Datasets/COCO dataset/COCO-Hand-splitdata/val/images/000000076170.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0556]
val: WARNING ⚠️ /home/nikhil/Documents/Datasets/COCO dataset/COCO-Hand-splitdata/val/images/000000096566.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0198]
val: WARNING ⚠️ /home/nikhil/Documents/Datasets/COCO dataset/COCO-Hand-splitdata/val/images/000000115521.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0583]
val: WARNING ⚠️ /home/nikhil/Documents/Datasets/COCO dataset/COCO-Hand-splitdata/val/images/000000126356.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0514]
val: WARNING ⚠️ /home/nik

OutOfMemoryError: CUDA out of memory. Tried to allocate 226.00 MiB (GPU 0; 23.65 GiB total capacity; 4.90 GiB already allocated; 150.62 MiB free; 5.12 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [12]:
#Train model using previous weights

def train_model(data_yaml, weights_path=None, additional_epochs=100, batch_size=16):
    """
    Train the YOLOv8 model on the COCO Big Hand dataset.
    """

    model = YOLO(weights_path)

    # Set the batch size for training
    model.batch_size = batch_size  

    # Continue training the model with the specified batch size
    model.train(data=data_yaml, epochs=additional_epochs, project="COCO-Hand", name="COCO_Hand_Big", imgsz=640)
    model.val()


In [13]:
if __name__ == "__main__":
    data_yaml_path = '/home/nikhil/Documents/Datasets/COCO dataset/COCO-Hand-splitdata/coco_hand_big.yaml'
    previous_weights_path = Path(os.path.expanduser("~/Documents/Training_weights/COCO_Trained model weights 45 epochs/COCO_Trained model weights 45 epochs /COCO_Hand_Big/weights/best.pt"))

    # Continue training for additional epochs using the weights from the last checkpoint
    train_model(data_yaml_path, weights_path=previous_weights_path, additional_epochs=100,batch_size=16)


Ultralytics YOLOv8.0.208 🚀 Python-3.9.7 torch-1.13.1 CUDA:0 (NVIDIA TITAN RTX, 24220MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
engine/trainer: task=detect, mode=train, model=/home/nikhil/Documents/Training_weights/COCO_Trained model weights 45 epochs/COCO_Trained model weights 45 epochs /COCO_Hand_Big/weights/best.pt, data=/home/nikhil/Documents/Datasets/COCO dataset/COCO-Hand-splitdata/coco_hand_big.yaml, epochs=100, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=COCO-Hand, name=COCO_Hand_Big, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, sa

val: Scanning /home/nikhil/Documents/Datasets/COCO dataset/COCO-Hand-splitdata/val/labels.cache... 2650 images, 0 backgrounds, 15 corrupt: 100%|██████████| 2650/2650 [00:00<?, ?it/s]
val: WARNING ⚠️ /home/nikhil/Documents/Datasets/COCO dataset/COCO-Hand-splitdata/val/images/000000076170.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0556]
val: WARNING ⚠️ /home/nikhil/Documents/Datasets/COCO dataset/COCO-Hand-splitdata/val/images/000000096566.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0198]
val: WARNING ⚠️ /home/nikhil/Documents/Datasets/COCO dataset/COCO-Hand-splitdata/val/images/000000115521.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0583]
val: WARNING ⚠️ /home/nikhil/Documents/Datasets/COCO dataset/COCO-Hand-splitdata/val/images/000000126356.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0514]
val: WARNING ⚠️ /home/nik


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     11/100      11.6G       1.65      1.035      1.269         32        640: 100%|██████████| 1313/1313 [10:52<00:00,  2.01it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 83/83 [00:13<00:00,  6.02it/s]
                   all       2635       4928      0.807      0.675      0.744      0.399

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     12/100      11.6G      1.643      1.026      1.261         29        640: 100%|██████████| 1313/1313 [10:52<00:00,  2.01it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 83/83 [00:13<00:00,  6.00it/s]
                   all       2635       4928      0.823      0.684       0.75        0.4

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     13/100      11.6G      1.633      1.019   

     48/100      11.6G      1.474     0.8636      1.175         37        640: 100%|██████████| 1313/1313 [10:57<00:00,  2.00it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 83/83 [00:13<00:00,  6.08it/s]
                   all       2635       4928      0.821       0.69      0.758      0.411

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     49/100      11.6G      1.468     0.8537      1.173         35        640: 100%|██████████| 1313/1313 [10:56<00:00,  2.00it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 83/83 [00:13<00:00,  6.16it/s]
                   all       2635       4928       0.82      0.691      0.758      0.411

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     50/100      11.6G      1.463     0.8524      1.172         33        640: 100%|██████████| 1313/1313 [10:57<00:00,  2.00i

lr/pg0,▁▅█████▇▇▇▇▇▇▇▇▆▆▆▆▆▆▆▅▅▅▅▅▅▅▄▄▄▄▄▄▄▃▃▃▃
lr/pg1,▁▅█████▇▇▇▇▇▇▇▇▆▆▆▆▆▆▆▅▅▅▅▅▅▅▄▄▄▄▄▄▄▃▃▃▃
lr/pg2,▁▅█████▇▇▇▇▇▇▇▇▆▆▆▆▆▆▆▅▅▅▅▅▅▅▄▄▄▄▄▄▄▃▃▃▃
metrics/mAP50(B),▆▆▁▄▄▃▄▅▅▆▆▆▆▅▅▇▆▇▆▇▇▇▇▆▇▇▇▇▇██▇███████▆
metrics/mAP50-95(B),██▁▃▄▃▄▄▄▅▅▄▅▅▆▆▆▆▆▆▇▇▇▇▆▇▇▆▇▇▇▇▇▇▇▇▇▇▇█
metrics/precision(B),▇▇▄▄▄▁▄▅▂▇▁▄▁▅▂▅▁▄▄▄▃▃▅▆▇█▅▂▂█▇▅▆▇▅▆▆▆▆█
metrics/recall(B),▅▄▁▂▃▄▃▄▄▆▆▆▇▅▇▆▇▇▆██▇▇▅▇▆▇█▇▇▇▇▇▇█▇▇▇▇▅
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/box_loss,▆▆▇██▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▁


Ultralytics YOLOv8.0.208 🚀 Python-3.9.7 torch-1.13.1 CUDA:0 (NVIDIA TITAN RTX, 24220MiB)
Model summary (fused): 218 layers, 25840339 parameters, 0 gradients, 78.7 GFLOPs
val: Scanning /home/nikhil/Documents/Datasets/COCO dataset/COCO-Hand-splitdata/val/labels.cache... 2650 images, 0 backgrounds, 15 corrupt: 100%|██████████| 2650/2650 [00:00<?, ?it/s]
val: WARNING ⚠️ /home/nikhil/Documents/Datasets/COCO dataset/COCO-Hand-splitdata/val/images/000000076170.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0556]
val: WARNING ⚠️ /home/nikhil/Documents/Datasets/COCO dataset/COCO-Hand-splitdata/val/images/000000096566.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0198]
val: WARNING ⚠️ /home/nikhil/Documents/Datasets/COCO dataset/COCO-Hand-splitdata/val/images/000000115521.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0583]
val: WARNING ⚠️ /home/nikhil/Documents/Datasets/COCO 

In [ ]:
#Uploading weights to drive
def upload_file_to_drive(source_path, destination_path):
    """
    Uploads a file from the Colab environment to Google Drive.

    Parameters:
    - source_path: The path to the file in the Colab environment.
    - destination_path: The path in Google Drive where the file will be stored.
    """
    # Ensure the destination directory exists
    os.makedirs(os.path.dirname(destination_path), exist_ok=True)

    # Copy the file to the destination directory
    shutil.copy(source_path, destination_path)
    print(f"File {source_path} uploaded to {destination_path}")


upload_file_to_drive('/content/COCO-Hand', '/content/drive/Shareddrives/Nikhil shared drive')


In [ ]:
#Testing model starts from here
#Loading the weights
!cp -r "/content/drive/Shareddrives/Nikhil shared drive/COCO_Trained model weights 45 epochs /COCO_Hand_Big/weights/best.pt" "/content"

In [ ]:
#plotting the bounding coordinates on images
def plot_one_box(xyxy, img, label=None, color=None, line_thickness=None):
    """
    Plots one bounding box on an image using OpenCV.

    Args:
    - xyxy (list): List of 4 values representing the bounding box coordinates [x1, y1, x2, y2].
    - img (numpy.ndarray): Image on which to draw the bounding box.
    - label (str, optional): Label to display above the bounding box.
    - color (tuple, optional): Color of the bounding box. Default is red.
    - line_thickness (int, optional): Thickness of the bounding box lines.

    Returns:
    - numpy.ndarray: Image with the bounding box drawn.
    """
    color = color or [0, 0, 255]  # Default color is red
    line_thickness = line_thickness or 2  # Default line thickness is 2
    c1, c2 = (int(xyxy[0]), int(xyxy[1])), (int(xyxy[2]), int(xyxy[3]))
    cv2.rectangle(img, c1, c2, color, thickness=line_thickness)
    if label:
        t_size = cv2.getTextSize(label, 0, fontScale=0.5, thickness=1)[0]
        c2 = c1[0] + t_size[0], c1[1] - t_size[1] - 3
        cv2.rectangle(img, c1, c2, color, -1)  # Negative thickness means filled rectangle
        cv2.putText(img, label, (c1[0], c1[1] - 2), 0, 0.5, [225, 255, 255], thickness=1, lineType=cv2.LINE_AA)
    return img


In [ ]:
#Testing the model
def test_model(weights_path, image_dir, show=False):
    """
    Test the trained YOLOv8 model on a set of images.
    """
    image_dir = Path(image_dir)
    model = YOLO(weights_path)

    for image_path in image_dir.iterdir():
        image = cv2.imread(image_path.as_posix())
        results = model.predict(source=image, conf=0.3, iou=0.4)

        # Extract the bounding boxes
        detections = results[0].boxes.xyxy

        # Attempt to extract confidence scores from the boxes
        try:
            confidences = [box[4].item() for box in detections]
        except IndexError:
            confidences = [1.0] * len(detections)  # Default to confidence of 1.0 if not available

        # Check if there are any detections
        if detections is not None:
            for i, box in enumerate(detections):
                xyxy = box[:4].tolist()
                conf = confidences[i]
                label = f'hand'  # Assuming only 'hand' class
                plot_one_box(xyxy, image, label=label, color=(255, 0, 0), line_thickness=2)

            if show:
                plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
                plt.show()
        else:
            print(f"No results for image {image_path.name}!")


In [ ]:
weights_path = "/content/COCO_Trained model/COCO_Hand_Big/best (1).pt"
test_images_path = "/content/drive/Shareddrives/Nikhil shared drive/hand14k/Images"
test_model(weights_path, test_images_path, show=True)

Preprocessing the images, masks by cropping

In [ ]:
zip_file_path = '/content/drive/Shareddrives/Nikhil shared drive/GTEA Cropped Dataset 6.6k Images/Cropped_Images.zip'
extraction_path = '/content/data/'

In [ ]:
# Unzip the file
with ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extraction_path)

print("Unzipped successfully!")

In [ ]:
def crop_mask_image(mask_dir, box, original_filename, scale=1.2):
    """
    Crop the mask image using the same bounding box coordinates from the original image.

    Args:
    - mask_dir (str): Directory where the mask images are stored.
    - box (list): List of 4 values representing the bounding box coordinates [x1, y1, x2, y2].
    - original_filename (str): Filename of the original image to find the corresponding mask image.
    - scale (float): Factor to scale the bounding box size.

    Returns:
    - numpy.ndarray: Cropped mask image.
    """
    # Replace the extension of the original filename with .png
    mask_filename = Path(original_filename).stem + '.png'

    # Construct the full path for the mask image
    mask_image_path = Path(mask_dir) / mask_filename
    # Read the mask image
    mask_image = cv2.imread(str(mask_image_path), cv2.IMREAD_UNCHANGED)
    if mask_image is None:
        raise FileNotFoundError(f"The mask image at {mask_image_path} was not found.")

    # Calculate the size of the square bounding box
    x1, y1, x2, y2 = box
    width = x2 - x1
    height = y2 - y1
    side_length = max(width, height) * scale

    # Calculate the center of the original bounding box
    center_x = x1 + width / 2
    center_y = y1 + height / 2

    # Calculate the new bounding box coordinates
    new_x1 = max(int(center_x - side_length / 2), 0)
    new_y1 = max(int(center_y - side_length / 2), 0)
    new_x2 = min(int(center_x + side_length / 2), mask_image.shape[1])
    new_y2 = min(int(center_y + side_length / 2), mask_image.shape[0])

    # Crop the mask image
    cropped_mask = mask_image[new_y1:new_y2, new_x1:new_x2]

    return cropped_mask


In [ ]:
def save_cropped_image(image, save_dir, original_filename):
    """
    Save the cropped image to the specified directory with the original filename.

    Args:
    - image (numpy.ndarray): The cropped image to save.
    - save_dir (Path): The directory to save the cropped image.
    - original_filename (str): The original filename for the cropped image.

    Returns:
    - str: The path to the saved image.
    """
    # Create the directory if it doesn't exist
    save_dir.mkdir(parents=True, exist_ok=True)

    # Save the image
    save_path = save_dir / original_filename
    cv2.imwrite(str(save_path), image)

    return str(save_path)


In [ ]:
#Cropping the images with (1.2 * s, 1.2 * s) scale
import cv2
from pathlib import Path
import matplotlib.pyplot as plt

def crop_and_resize(image, box, scale=1.2):
    """
    Crop and resize the bounding box in the image.

    Args:
    - image (numpy.ndarray): The image from which to crop.
    - box (list): The bounding box coordinates [x1, y1, x2, y2].
    - scale (float): Factor to scale the bounding box size.

    Returns:
    - numpy.ndarray: The cropped and resized image.
    """
    # Calculate width and height of the box
    width = box[2] - box[0]
    height = box[3] - box[1]
    # Determine the size of the square
    side_length = max(width, height)
    # Calculate the new box coordinates, centered around the original box, with the new size
    center_x, center_y = box[0] + width / 2, box[1] + height / 2
    new_half_size = side_length * scale / 2
    new_x1 = max(center_x - new_half_size, 0)
    new_y1 = max(center_y - new_half_size, 0)
    new_x2 = min(center_x + new_half_size, image.shape[1])
    new_y2 = min(center_y + new_half_size, image.shape[0])
    # Crop the image
    cropped_img = image[int(new_y1):int(new_y2), int(new_x1):int(new_x2)]
    return cropped_img

def test_model(weights_path, image_dir, mask_dir, show=False):
    """
    Test the trained YOLOv8 model on a set of images, crop the hand bounding boxes from both the images and the corresponding masks,
    and save them to separate directories.
    """
    image_dir = Path(image_dir)
    mask_dir = Path(mask_dir)
    cropped_images_dir = Path('Cropped_Images')
    cropped_masks_dir = Path('Cropped_Masks')
    model = YOLO(weights_path)
    hand_detection_count = 0

    for image_path in image_dir.iterdir():
        image = cv2.imread(str(image_path))
        results = model.predict(source=image, conf=0.3, iou=0.4)

        # Extract the bounding boxes
        detections = results[0].boxes.xyxy

        # Check if there are any detections
        if detections is not None and len(detections) > 0:
            hand_detection_count += 1  # Increment the counter for each image with a detection
            for i, box in enumerate(detections):
                xyxy = box[:4].tolist()
                cropped_image = crop_and_resize(image, xyxy)
                cropped_mask = crop_mask_image(mask_dir, xyxy, image_path.name)

                # Save the cropped image and mask
                cropped_image_path = save_cropped_image(cropped_image, cropped_images_dir, image_path.name)
                cropped_mask_path = save_cropped_image(cropped_mask, cropped_masks_dir, image_path.name)

                '''if show:
                    # Show the cropped image and mask side by side
                    fig, axs = plt.subplots(1, 2, figsize=(10, 5))
                    axs[0].imshow(cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB))
                    axs[0].set_title('Cropped Image')
                    axs[1].imshow(cropped_mask, cmap='gray')
                    axs[1].set_title('Cropped Mask')
                    plt.show()'''

        else:
            print(f"No results for image {image_path.name}")
    print(f"Total number of images with detected hands: {hand_detection_count}")




In [ ]:
weights_path = "/content/drive/Shareddrives/Nikhil shared drive/COCO_Hand_Big2/weights/best.pt"
test_images_path = "/content/data/Images"
mask_dir="/content/data/Masks"
test_model(weights_path, test_images_path,mask_dir, show=True)

In [ ]:
def smooth_mask(mask, blur_size=3):
    """
    Apply Gaussian blur to the mask to smooth the edges.

    Args:
    - mask (numpy.ndarray): The mask image.
    - blur_size (int): Size of the kernel to use for the Gaussian blur.

    Returns:
    - numpy.ndarray: The smoothed mask.
    """
    return cv2.GaussianBlur(mask, (blur_size, blur_size), 0)


In [ ]:
#To convert the masked images into labels txt file
import cv2
import numpy as np
from pathlib import Path

def mask_to_polygons(mask):
    """
    Convert a mask image to a list of polygons.

    Args:
    - mask (numpy.ndarray): The mask image.

    Returns:
    - list: A list of polygons, each represented as a list of points (x, y).
    """
    # Find contours in the mask
    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    polygons = []
    for contour in contours:
        # Approximate the contour to a polygon
        epsilon = 0.005 * cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, True)
        polygon = approx.ravel().tolist()
        # Skip small or invalid contours
        if len(polygon) >= 6:
            polygons.append(polygon)
    return polygons





In [ ]:
def save_polygons_to_txt(polygons, save_path):
    """
    Save the polygon coordinates to a .txt file.

    Args:
    - polygons (list): A list of polygons.
    - save_path (Path): The path to the .txt file where the coordinates will be saved.
    """
    with open(save_path, 'w') as file:
        for polygon in polygons:
            coordinates = ','.join(map(str, polygon))
            file.write(coordinates + '\n')

In [ ]:
#Converting masks into labels
def process_masks_to_polygons(masks_dir, labels_dir):
    """
    Process all mask images in a directory, convert them to polygons, and save the results to .txt files.

    Args:
    - masks_dir (str): Directory where the mask images are stored.
    - labels_dir (str): Directory where the .txt files will be saved.
    """
    masks_dir = Path(masks_dir)
    labels_dir = Path(labels_dir)
    labels_dir.mkdir(parents=True, exist_ok=True)

    for mask_path in masks_dir.glob('*.jpg'):
        mask = cv2.imread(str(mask_path), cv2.IMREAD_GRAYSCALE)
        if mask is None:
            continue
        mask = smooth_mask(mask)
        polygons = mask_to_polygons(mask)
        txt_filename = mask_path.stem + '.txt'
        save_path = labels_dir / txt_filename
        save_polygons_to_txt(polygons, save_path)

masks_dir = '/content/data/Cropped_Masks'
labels_dir = '/content/data/Cropped_Labels'
process_masks_to_polygons(masks_dir, labels_dir)


In [ ]:
#Plotting the LABELS on Images and placing in folder
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

# Function to read polygons from a text file
def read_polygons_from_file(file_path):
    polygons = []
    with open(file_path, 'r') as file:
        for line in file:
            points = line.strip().split(',')
            polygon = list(zip(map(int, points[::2]), map(int, points[1::2])))
            polygons.append(polygon)
    return polygons

# Paths to the folders containing images and labels
images_folder_path = '/content/drive/Shareddrives/Nikhil shared drive/GTEA Cropped Dataset 6.6k Images/Cropped_Images'  # Replace with your images folder path
labels_folder_path = '/content/data/Cropped_Labels'  # Replace with your labels folder path
output_folder_path = '/content/data/Cropped_LabelsonImages'

# Create the output folder if it doesn't exist
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

# Get a list of image files and corresponding label files
image_files = [f for f in os.listdir(images_folder_path) if f.endswith('.jpg')]
label_files = [f.replace('.jpg', '.txt') for f in image_files]

# Loop through the image and label files
for image_file, label_file in zip(image_files, label_files):
    image_path = os.path.join(images_folder_path, image_file)
    label_path = os.path.join(labels_folder_path, label_file)
    output_image_path = os.path.join(output_folder_path, image_file)
    # Read polygons from the text file
    polygons = read_polygons_from_file(label_path)

    # Load the image
    image = cv2.imread(image_path)

    # Check if the image was successfully loaded
    if image is None:
        print(f"Error: Image {image_file} not found.")
        continue

    # Convert image to RGB (OpenCV uses BGR by default)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Draw the polygons on the image
    for polygon in polygons:
        pts = np.array(polygon, np.int32)
        pts = pts.reshape((-1, 1, 2))
        cv2.polylines(image, [pts], isClosed=True, color=(0, 255, 0), thickness=2)

    # Save the image with polygons
    cv2.imwrite(output_image_path, cv2.cvtColor(image, cv2.COLOR_RGB2BGR))

print("Processing complete. Images with labeled polygons are saved in 'Cropped_LabelsonImages' folder.")

'''# Display the image with polygons
    plt.imshow(image)
    plt.title(f'Polygons on {image_file}')
    plt.axis('off')  # Hide the axis
    plt.show()'''


In [ ]:
#To download folder into local system
import os
from google.colab import files
from zipfile import ZipFile

def zip_folder(folder_path, zip_path):
    """
    Zip the contents of an entire folder (with that folder included in the archive).
    Existing files with the same name will be overwritten.

    Args:
    - folder_path (str): The path to the folder to zip.
    - zip_path (str): The complete target path for the created zip file.
    """
    with ZipFile(zip_path, 'w') as zipf:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                # Create a relative path for files to keep the directory structure
                relative_path = os.path.relpath(os.path.join(root, file), os.path.join(folder_path, '..'))
                zipf.write(os.path.join(root, file), relative_path)

def download_folder(folder_path):
    """
    Compress and download the given folder.

    Args:
    - folder_path (str): The path to the folder to download.
    """
    zip_path = f"{folder_path}.zip"
    zip_folder(folder_path, zip_path)
    files.download(zip_path)

# Example usage:
folder_to_download = '/content/data/Cropped_Labels'  # Change this to the path of the folder you want to download
download_folder(folder_to_download)

In [ ]:
#to count number of images in a directory
import os

def count_images_in_folder(folder_path):
    # List all files in the folder
    files = os.listdir(folder_path)
    # Count files with image extensions (assuming common image formats)
    image_count = sum(1 for file in files if file.lower().endswith(('.png', '.jpg', '.jpeg', '.txt', '.gif', '.tiff')))
    return image_count

folder_path = '/content/data/Cropped_LabelsonImages'  # Replace with your folder path
print(f"Number of images in the folder: {count_images_in_folder(folder_path)}")


In [ ]:
#renaming files in folder
import os

# Define the directory where your images are located
directory = "/content/drive/Shareddrives/Nikhil shared drive/Cropped_Masks"  # Adjust the path to your specific directory

# Iterate over all files in the directory
for filename in os.listdir(directory):
    if filename.startswith("Mask_"):  # Check if the filename starts with "Image_"
        # Construct the old file path
        old_file = os.path.join(directory, filename)

        # Construct the new file path with "Image_" removed from the filename
        new_file = os.path.join(directory, filename.replace("Mask_", ""))

        # Rename the file
        os.rename(old_file, new_file)
        print(f'Renamed: {old_file} to {new_file}')


In [ ]:
#To clear entire directory contents
import shutil
import os

def clear_directory_contents(dir_path):
    """
    Remove all files and subdirectories in the specified directory.

    Args:
    - dir_path (str): The path to the directory to clear.
    """
    for filename in os.listdir(dir_path):
        file_path = os.path.join(dir_path, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print(f'Failed to delete {file_path}. Reason: {e}')

# Example usage:
directory_to_clear = '/content/drive/Shareddrives/Nikhil shared drive/Cropped_Masks'  # Replace with your directory path
clear_directory_contents(directory_to_clear)


In [ ]:
#To upload folder from colab to drive
import shutil
from pathlib import Path

def upload_file_to_drive(source, destination):
    source_path = Path(source)
    destination_path = Path(destination)

    # Check if the source is a file or directory
    if source_path.is_file():
        # Copy the file
        shutil.copy2(source, destination)
    elif source_path.is_dir():
        # Copy the directory
        destination_path = destination_path / source_path.name
        shutil.copytree(source, destination_path)
    else:
        print(f"The source {source} is neither a file nor a directory.")


upload_file_to_drive('/content/Cropped_LabelsonImages', '/content/drive/MyDrive')